In [1]:
import pandas as pd
import numpy as np
import nltk

## Preprocessamento
Nesta etapa será definida funções para limpar os dados, além de remover as stopwords

In [70]:
train = pd.read_excel('datasets/emotions-pt.xlsx')
train.head()

,conteúdo,sentimento
0,eu não me senti humilhado,tristeza
1,Eu posso passar de uma sensação de desesperanç...,tristeza
2,estou pegando um minuto para postar me sinto g...,raiva
3,estou me sentindo mal-humorado,raiva
4,Eu tenho me sentido um pouco sobrecarregado ul...,tristeza


In [63]:
train['sentimento'].value_counts()

alegria     5362
tristeza    4666
raiva       2159
Name: sentimento, dtype: int64

In [71]:
print(f"Antes: {train['conteúdo'].count()}")

train.drop_duplicates(inplace=True)
print(f"Depois: {train['conteúdo'].count()}")

Antes: 12187
Depois: 12172


In [72]:
#Separando as sentencas dos sentimentos
X_train = train['conteúdo']
Y_train = train['sentimento']

In [83]:
#É necessário baixar as stopwords
nltk.download('stopwords')

#Criação da função que remove stopwords
def remover_stopwords(conteudo):
  stopwords = set(nltk.corpus.stopwords.words('portuguese'))
  palavras = [p.lower() for p in conteudo.split() if p.lower() not in stopwords]
  return " ".join(palavras)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
#Criação da função de Stemming, para reduzir as palavras em radicais
def criar_radicais(conteudo):
  stemmer = nltk.stem.RSLPStemmer()
  palavras = [stemmer.stem(p) for p in conteudo.split()]
  return " ".join(palavras)

In [84]:
def preprocessamento(instancia):
  instancia = remover_stopwords(instancia)
  instancia = re.sub("[,.;/\[\]?!\'\"]", '', instancia)
  return instancia


In [85]:
#Realizando o preprocessamento nos dados
X_train = X_train.apply(lambda x: preprocessamento(x))
X_train.head()

0                                      senti humilhado
1    posso passar sensação desesperança maldita esp...
2        pegando minuto postar sinto ganancioso errado
3                                sentindo mal-humorado
4    sentido pouco sobrecarregado ultimamente certe...
Name: conteúdo, dtype: object

## Classificação
Será utilizado SVC e Naivy Bayes (MultinomialNB) para realizar o fit dos dados, além da tokenização dos dados. Por fim, será validado os modelos com o Cross Validation.

In [91]:
#Criando um pipeline para Tokenização e Classificação
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

#Pipelina para Naivy Bayes
pipe_nb = Pipeline([
    ('vetor', CountVectorizer(analyzer='word')),
    ('nb', MultinomialNB())
])

#Pipeline para SVC
pipe_svc = Pipeline([
    ('vetor', CountVectorizer(analyzer='word')),
    ('svc', SVC(kernel='linear'))
])

In [97]:
from sklearn.model_selection import cross_val_score

score_nb = cross_val_score(pipe_nb, X_train, Y_train, cv=5)
score_svc = cross_val_score(pipe_svc, X_train, Y_train, cv=5)

In [101]:
print(f'MultinomialNB: {round(score_nb.mean(), 3)}\nSVC: {round(score_svc.mean(), 3)}')

MultinomialNB: 0.827
SVC: 0.857
